In [1]:
import talos as ta
import gensim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import tensorflow_addons as tfa

from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Flatten, Dense, TimeDistributed, \
    SpatialDropout1D, Bidirectional, Conv1D, MaxPooling1D, GlobalAveragePooling1D, GlobalMaxPooling1D
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import load_model
from tensorflow.keras.metrics import Precision, Recall
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import Callback
from tensorflow.keras.preprocessing.text import Tokenizer

from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, precision_score, recall_score, classification_report

%matplotlib inline

Using TensorFlow backend.


In [2]:
# class ConfusionMatrixMetric(tf.keras.metrics.Metric):
#     def __init__(self, num_classes, **kwargs):
#         super(ConfusionMatrixMetric,self).__init__(name='confusion_matrix_metric',**kwargs)
#         self.num_classes=num_classes
#         self.total_cm = self.add_weight("total", shape=(num_classes,num_classes), initializer="zeros")
        
#     def reset_states(self):
#         for s in self.variables:
#             s.assign(tf.zeros(shape=s.shape))
            
#     def update_state(self, y_true, y_pred,sample_weight=None):
#         self.total_cm.assign_add(self.confusion_matrix(y_true,y_pred))
#         return self.total_cm
        
#     def result(self):
#         return self.process_confusion_matrix()
    
#     def confusion_matrix(self,y_true, y_pred):
#         y_pred = tf.argmax(y_pred, axis=2)
#         y_true = tf.argmax(y_true, axis=2)

#         y_pred = tf.reshape(y_pred, [-1])
#         y_true = tf.reshape(y_true, [-1])
  
#         cm = tf.math.confusion_matrix(
#             y_true, 
#             y_pred, 
#             dtype=tf.float32, 
#             num_classes=self.num_classes
#         )
        
#         return cm
    
#     def process_confusion_matrix(self):
#         cm = self.total_cm
#         diag_part=tf.linalg.diag_part(cm)
#         precision=diag_part/(tf.reduce_sum(cm,0)+tf.constant(1e-15))
#         recall=diag_part/(tf.reduce_sum(cm,1)+tf.constant(1e-15))
#         f1=2*precision*recall/(precision+recall+tf.constant(1e-15))
#         return precision, recall, f1
    
#     def fill_output(self,output):
#         results=self.result()
#         for i in range(self.num_classes):
#             output['precision_{}'.format(i)]=results[0][i]
#             output['recall_{}'.format(i)]=results[1][i]
#             output['f1_{}'.format(i)]=results[2][i]

In [3]:
class SeqModel(tf.keras.Sequential):
    def train_step(self, data):
        x, y = data
        
        with tf.GradientTape() as tape:
            y_pred = self(x, training=True)
            loss = self.compiled_loss(
                y,
                y_pred,
                regularization_losses=self.losses,
            )

        # Compute gradients
        trainable_vars = self.trainable_variables
        gradients = tape.gradient(loss, trainable_vars)

        # Update weights
        self.optimizer.apply_gradients(zip(gradients, trainable_vars))
        self.compiled_metrics.update_state(y, y_pred)
        output={m.name: m.result() for m in self.metrics[:-1]}
        
        if 'confusion_matrix_metric' in self.metrics_names:
            self.metrics[-1].fill_output(output)

        return output
        
        
    def test_step(self, data):
        x, y = data

        y_pred = self(x, training=False)
        loss = self.compiled_loss(
            y,
            y_pred,
            regularization_losses=self.losses,
        )

        self.compiled_metrics.update_state(y, y_pred)
        output={m.name: m.result() for m in self.metrics[:-1]}
        
        if 'confusion_matrix_metric' in self.metrics_names:
            self.metrics[-1].fill_output(output)    
        
        return output

In [4]:
df = pd.read_csv('../data/data.csv')

In [5]:
df.head()

,word,tag
0,Pengamat,O
1,politik,O
2,dari,O
3,Universitas,B-ORGANIZATION
4,Gadjah,I-ORGANIZATION


In [6]:
sentences = []
cnt = 1

for i in df.itertuples():
    sentences.append(cnt)
    
    if '.' in str(i.word):
        cnt += 1
        
df['sentence #'] = sentences
df.head()

,word,tag,sentence #
0,Pengamat,O,1
1,politik,O,1
2,dari,O,1
3,Universitas,B-ORGANIZATION,1
4,Gadjah,I-ORGANIZATION,1


In [7]:
agg_func = lambda s: [(w, t) for w, t in zip(s['word'].values.tolist(), s['tag'].values.tolist())]
grouped = df.groupby('sentence #').apply(agg_func)
sentences = [s for s in grouped]

In [8]:
words = list(set(df['word'].values))
words.append('PADDING')
num_words = len(words)
tags = list(set(df['tag'].values))
num_tags = len(tags)

In [9]:
word2idx = {w: i for i, w in enumerate(words)}
tag2idx = {t: i for i, t in enumerate([tag for tag in tags if tag != 'O'])}
tag2idx['O'] = len(tags)-1

In [10]:
tag2idx

{'I-TIME': 0,
 'B-TIME': 1,
 'I-PERSON': 2,
 'I-ORGANIZATION': 3,
 'B-ORGANIZATION': 4,
 'U-LOCATION': 5,
 'U-QUANTITY': 6,
 'U-ORGANIZATION': 7,
 'L-PERSON': 8,
 'B-LOCATION': 9,
 'U-PERSON': 10,
 'B-QUANTITY': 11,
 'L-ORGANIZATION': 12,
 'B-PERSON': 13,
 'U-TIME': 14,
 'I-QUANTITY': 15,
 'L-TIME': 16,
 'L-QUANTITY': 17,
 'I-LOCATION': 18,
 'L-LOCATION': 19,
 'O': 20}

In [11]:
num_words

13031

In [12]:
max_len = 80
X = [[word2idx[w[0]] for w in s] for s in sentences]
X = pad_sequences(maxlen=max_len, sequences=X, padding='post', value=num_words-1)

y = [[tag2idx[w[1]] for w in s] for s in sentences]
y = pad_sequences(maxlen=max_len, sequences=y, padding='post', value=tag2idx['O'])
y = [to_categorical(i, num_classes=num_tags) for i in y]

In [13]:
np.array(y).shape

(4892, 80, 21)

In [14]:
x_train, x_tmp, y_train, y_tmp = train_test_split(X, y, test_size=0.3, random_state=1)
x_val, x_test, y_val, y_test = train_test_split(x_tmp, y_tmp, test_size=0.5, random_state=1)

In [15]:
model = gensim.models.Word2Vec.load("../checkpoint/w2vec_wiki_id_case")

In [16]:
embedding_matrix = np.zeros((len(words), 400))

for i, w in enumerate(words):
    try:
        embedding_vector = model.wv[w]
        embedding_matrix[i] = embedding_vector
    except:
        embedding_matrix[i] = np.random.normal(0, np.sqrt(0.25), 400)

In [31]:
params = {
    'optimizer': ['Adam'],
    'lr': [0.01],
    'units': [50],
    'dropout': [0.5],
    'batch_size': [64, 128, 256]
}

In [32]:
def create_model(x_train, y_train, x_val, y_val, params):
    model = SeqModel([
        Input(shape=(max_len, )),
        Embedding(
            input_dim= embedding_matrix.shape[0], 
            weights=[embedding_matrix], 
            output_dim=embedding_matrix.shape[1], 
            input_length=max_len,
            trainable=False
        ),
        SpatialDropout1D(params['dropout']),
        Bidirectional(LSTM(units=params['units'], return_sequences=True, recurrent_dropout=params['dropout'])),
        Dense(num_tags, activation='softmax')
    ])

    f1_score_m = tfa.metrics.F1Score(
        num_classes=num_tags,
        average='micro',
        name='f1_score',
        threshold=0.5
    )
    
    if params['optimizer'] == 'Nadam':
        optm = tf.keras.optimizers.Nadam(lr=params['lr'], )
        
    if params['optimizer'] == 'Adam':
        optm = tf.keras.optimizers.Adam(lr=params['lr'])
        
    if params['optimizer'] == 'RMSprop':
        optm = tf.keras.optimizers.RMSprop(lr=params['lr'])
    
    model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(), 
        optimizer=optm,
        metrics=[f1_score_m]
    )
    
    es = tf.keras.callbacks.EarlyStopping(patience=10)

    history = model.fit(
        x_train, np.array(y_train),
        validation_data=(x_val, np.array(y_val)),
        epochs=100, verbose=1, callbacks=[es], batch_size=params['batch_size']
    )
    
    return history, model

In [33]:
t = ta.Scan(x=x_train,
            y=np.array(y_train),
            x_val=x_val,
            y_val=np.array(y_val),
            model=create_model,
            params=params,
            experiment_name='bilstm_w2v_opt_v2', 
            val_split=None,
)

  0%|          | 0/4 [00:00<?, ?it/s]

Train on 3424 samples, validate on 734 samples
Epoch 1/100
3424/3424 [==============================] - 14s 4ms/sample - loss: 0.2253 - f1_score: 0.9542 - val_loss: 0.1435 - val_f1_score: 0.9657
Epoch 2/100
3424/3424 [==============================] - 12s 4ms/sample - loss: 0.1204 - f1_score: 0.9683 - val_loss: 0.0904 - val_f1_score: 0.9743
Epoch 3/100
3424/3424 [==============================] - 12s 3ms/sample - loss: 0.0902 - f1_score: 0.9738 - val_loss: 0.0748 - val_f1_score: 0.9780
Epoch 4/100
3424/3424 [==============================] - 12s 3ms/sample - loss: 0.0784 - f1_score: 0.9764 - val_loss: 0.0675 - val_f1_score: 0.9794
Epoch 5/100
3424/3424 [==============================] - 13s 4ms/sample - loss: 0.0716 - f1_score: 0.9781 - val_loss: 0.0631 - val_f1_score: 0.9807
Epoch 6/100
3424/3424 [==============================] - 16s 5ms/sample - loss: 0.0669 - f1_score: 0.9791 - val_loss: 0.0633 - val_f1_score: 0.9808
Epoch 7/100
3424/3424 [==============================] - 15s 4ms/

 25%|██▌       | 1/4 [07:26<22:20, 446.70s/it]

Train on 3424 samples, validate on 734 samples
Epoch 1/100
3424/3424 [==============================] - 13s 4ms/sample - loss: 0.2856 - f1_score: 0.9448 - val_loss: 0.1786 - val_f1_score: 0.9607
Epoch 2/100
3424/3424 [==============================] - 11s 3ms/sample - loss: 0.1623 - f1_score: 0.9632 - val_loss: 0.1337 - val_f1_score: 0.9666
Epoch 3/100
3424/3424 [==============================] - 10s 3ms/sample - loss: 0.1240 - f1_score: 0.9675 - val_loss: 0.0986 - val_f1_score: 0.9721
Epoch 4/100
3424/3424 [==============================] - 10s 3ms/sample - loss: 0.0992 - f1_score: 0.9718 - val_loss: 0.0825 - val_f1_score: 0.9760
Epoch 5/100
3424/3424 [==============================] - 10s 3ms/sample - loss: 0.0872 - f1_score: 0.9744 - val_loss: 0.0753 - val_f1_score: 0.9782
Epoch 6/100
3424/3424 [==============================] - 10s 3ms/sample - loss: 0.0788 - f1_score: 0.9766 - val_loss: 0.0694 - val_f1_score: 0.9793
Epoch 7/100
3424/3424 [==============================] - 10s 3ms/

 50%|█████     | 2/4 [15:10<15:03, 451.88s/it]

...

3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0375 - f1_score: 0.9875 - val_loss: 0.0542 - val_f1_score: 0.9842
Epoch 57/100
3424/3424 [==============================] - 8s 2ms/sample - loss: 0.0366 - f1_score: 0.9879 - val_loss: 0.0544 - val_f1_score: 0.9836
Epoch 58/100
3424/3424 [==============================] - 8s 2ms/sample - loss: 0.0371 - f1_score: 0.9876 - val_loss: 0.0564 - val_f1_score: 0.9834
Epoch 59/100
3424/3424 [==============================] - 8s 2ms/sample - loss: 0.0366 - f1_score: 0.9880 - val_loss: 0.0535 - val_f1_score: 0.9843
Epoch 60/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0367 - f1_score: 0.9878 - val_loss: 0.0568 - val_f1_score: 0.9831
Epoch 61/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0364 - f1_score: 0.9878 - val_loss: 0.0543 - val_f1_score: 0.9841


 75%|███████▌  | 3/4 [23:40<07:49, 469.39s/it]

Train on 3424 samples, validate on 734 samples
Epoch 1/100
3424/3424 [==============================] - 11s 3ms/sample - loss: 0.5010 - f1_score: 0.9039 - val_loss: 0.2218 - val_f1_score: 0.9588
Epoch 2/100
3424/3424 [==============================] - 7s 2ms/sample - loss: 0.2250 - f1_score: 0.9572 - val_loss: 0.2106 - val_f1_score: 0.9591
Epoch 3/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.2096 - f1_score: 0.9577 - val_loss: 0.1943 - val_f1_score: 0.9592
Epoch 4/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.1934 - f1_score: 0.9588 - val_loss: 0.1764 - val_f1_score: 0.9620
Epoch 5/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.1766 - f1_score: 0.9618 - val_loss: 0.1606 - val_f1_score: 0.9639
Epoch 6/100
3424/3424 [==============================] - 8s 2ms/sample - loss: 0.1623 - f1_score: 0.9635 - val_loss: 0.1466 - val_f1_score: 0.9649
Epoch 7/100
3424/3424 [==============================] - 7s 2ms/sample

3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0463 - f1_score: 0.9850 - val_loss: 0.0535 - val_f1_score: 0.9833
Epoch 57/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0462 - f1_score: 0.9851 - val_loss: 0.0530 - val_f1_score: 0.9836
Epoch 58/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0462 - f1_score: 0.9850 - val_loss: 0.0532 - val_f1_score: 0.9835
Epoch 59/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0462 - f1_score: 0.9851 - val_loss: 0.0525 - val_f1_score: 0.9835
Epoch 60/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0455 - f1_score: 0.9852 - val_loss: 0.0527 - val_f1_score: 0.9835
Epoch 61/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0459 - f1_score: 0.9849 - val_loss: 0.0530 - val_f1_score: 0.9838
Epoch 62/100
3424/3424 [==============================] - 9s 3ms/sample - loss: 0.0449 - f1_score: 0.9853 - val_loss: 0.0540 

100%|██████████| 4/4 [35:21<00:00, 530.33s/it]


In [34]:
t.data[['loss', 'f1_score', 'val_loss', 'val_f1_score', 'batch_size']] \
    .sort_values(by=['val_f1_score'], ascending=False)

,loss,f1_score,val_loss,val_f1_score,batch_size
1,0.035668,0.988037,0.054054,0.984472,64
3,0.039222,0.986979,0.051227,0.984379,256
2,0.036361,0.987768,0.054267,0.984085,128
0,0.037876,0.987528,0.056256,0.983888,32
